Import statements

In [1]:
import requests    # http://python-requests.org/
import json
import time

Create initial lists/dictionaries. **Thank you [Master League](https://masterleague.net/) for the data!**

In [ ]:
endpoints = {
    "heroes": "https://api.masterleague.net/heroes/?format=api",
    "maps": "https://api.masterleague.net/maps/?format=api",
    "regions": "https://api.masterleague.net/regions/?format=api",
    "patches": "https://api.masterleague.net/patches/?format=api",
    "teams": "https://api.masterleague.net/teams/?format=api",
    "players": "https://api.masterleague.net/players/?format=api",
    "tournaments": "https://api.masterleague.net/tournaments/?format=api",
    "matches": "https://api.masterleague.net/matches/?format=api",
    "calendar": "https://api.masterleague.net/calendar/?format=api"
}

How to use the API for unauthenticated users. https://api.masterleague.net/?format=api

In [2]:
API_ROOT = 'https://api.masterleague.net'
API_USER = None
API_PASS = None

session = requests.Session()

Get data from heroes API.

In [3]:
r = session.get(API_ROOT + '/heroes.json')
data = json.loads(r.text)
heroes = {}
next_page = data['next']

for hero in data['results']:
    heroes[hero['id']] = hero['name']

while data['next'] is not None:
    next_page = data['next']
    text_results = session.get(next_page)
    data = json.loads(text_results.text)
    
    next_page = data['next']
    for hero in data['results']:
        heroes[hero['id']] = hero['name']

Test call to match API

In [ ]:
match_response = session.get(API_ROOT + '/matches.json')
match_data = json.loads(match_response.text)
match_data

Test call to tournament API

In [ ]:
tournament_response = session.get(API_ROOT + '/tournaments.json')
tournament_data = json.loads(tournament_response.text)
#tournament_data

## Tournament Functions

In [4]:
def add_tournament_details(tournament_dictionary, page_data):
    for tournament in page_data:
        tournament_id = tournament['id']
        tournament_name = tournament['name']
        tournament_dictionary[tournament_id] = tournament_name
    return tournament_dictionary

def create_tournament_dictionary(api_session, API_ROOT):
    tournaments = {}
    next_tournament_url = API_ROOT + '/tournaments.json'

    while next_tournament_url is not None:
        text_results = api_session.get(next_tournament_url)
        tournament_data = json.loads(text_results.text)
        page_data = tournament_data['results']
        tournaments = add_tournament_details(tournaments, page_data)
        next_tournament_url = tournament_data['next']
    
    return tournaments
    

## Match Functions

In [5]:
def get_match_data(tournament_id, API_ROOT):
    session = requests.Session()
    matches = {}
    match_url = API_ROOT + '/matches.json'
    
    while match_url is not None:
        text_results = session.get(match_url)
        match_data = json.loads(text_results.text)
        #print(match_data)
        for match in match_data['results']:
            tournament = match['tournament']
            if tournament == tournament_id:
                match_id = match['id']
                matches[match_id] = match
        time.sleep(2)
        match_url = match_data['next']
        
    return matches
    

### Create tournament dictionary

In [6]:
API_ROOT = 'https://api.masterleague.net'
API_USER = None
API_PASS = None

session = requests.Session()

In [7]:
tournament_dict = create_tournament_dictionary(session, API_ROOT)

In [ ]:
tournament_dict

https://api.masterleague.net/?format=api

Get match data for match **32: HGC North America - Phase 1**

In [8]:
match_dict = get_match_data(32, API_ROOT)

In [9]:
match_dict

{1762: {'date': '2016-11-21T00:00:00Z',
  'drafts': [{'bans': [21, 51],
    'is_winner': True,
    'picks': [{'hero': 27, 'player': 144},
     {'hero': 55, 'player': 174},
     {'hero': 2, 'player': 176},
     {'hero': 15, 'player': 490},
     {'hero': 18, 'player': 194}],
    'team': 108},
   {'bans': [50, 57],
    'is_winner': False,
    'picks': [{'hero': 23, 'player': 485},
     {'hero': 33, 'player': 384},
     {'hero': 48, 'player': 380},
     {'hero': 14, 'player': 175},
     {'hero': 47, 'player': 382}],
    'team': 129}],
  'duration': 1038,
  'game': 1,
  'id': 1762,
  'map': 10,
  'patch': 14,
  'round': '1/8 Final',
  'series': 634,
  'series_format': 'Bo3',
  'stage': 145,
  'tournament': 32,
  'url': 'https://masterleague.net/match/1762/'},
 1763: {'date': '2016-11-21T00:00:00Z',
  'drafts': [{'bans': [42, 27],
    'is_winner': True,
    'picks': [{'hero': 50, 'player': 144},
     {'hero': 16, 'player': 490},
     {'hero': 54, 'player': 194},
     {'hero': 32, 'player': 1

sleep 5 seconds, more than 45 minutes to run. what is the minimum time we can sleep?
Start: 11:46 (sleep 1 second) - wrong. infinite loop.
Restarted: 11:52 (sleep 1 second) - got results error. add printout of error message (try/catch?)

Started at 6:49 PM. sleep 2 seconds. Still running at 6:59. Finished at 7:04 - 15 min.

download pickle. import pickle. pickle the match dictionary ASAP.

In [11]:
import pickle
pickle.dump( match_dict, open( "tournament_32_matches.p", "wb" ) )

Which fields are relevant? id, drafts['is_winner'], drafts[0-1]['picks'][0-4]['hero'], map, patch

In [27]:
relevant_data = {}

for item in match_dict:
    match = match_dict.get(item)
    #print(match)
    match_id = match['id']
    draft_info = tuple(match['drafts'])
    match_map = match['map']
    patch = match['patch']
    print("id: %d, draft: %s, map: %s, patch: %d" % (match_id, draft_info, match_map, patch))
    relevant_data[match_id] = {draft_info, match_map, patch}
    


id: 2048, draft: ({'picks': [{'hero': 60, 'player': 570}, {'hero': 19, 'player': 172}, {'hero': 16, 'player': 419}, {'hero': 39, 'player': 154}, {'hero': 30, 'player': 187}], 'is_winner': True, 'team': 147, 'bans': [57, 21]}, {'picks': [{'hero': 43, 'player': 163}, {'hero': 32, 'player': 465}, {'hero': 48, 'player': 143}, {'hero': 26, 'player': 145}, {'hero': 2, 'player': 183}], 'is_winner': False, 'team': 29, 'bans': [33, 11]}), map: 10, patch: 16


TypeError: unhashable type: 'dict'